# Let's explore "spaCy" with our "candida posts" 

##### from: 
/Users/eran/Galvanize_more_repositories/Support_group_MD/pitch_day/merged_W_id.csv

based on https://github.com/Erolino/modern-nlp-in-python/blob/master/executable/Modern_NLP_in_Python.ipynb

but refer to my version notebook in ../Galvanize_more_repositories/modern-nlp-in-python/executable/.. : http://localhost:8888/notebooks/Galvanize_more_repositories/modern-nlp-in-python/executable/Modern_NLP_in_Python_mine.ipynb#first-i-needed-to-install-spacy-using:

In [1]:
import pandas as pd

In [2]:
# paths to be used later for more ease:
from os.path import join as joinp
pitch_path='/Users/eran/Galvanize_more_repositories/Support_group_MD/pitch_day'  # thr directory with the final files of the project
process_path='/Users/eran/Galvanize_more_repositories/Support_group_MD/processing_files'  # the directory with all the files used for NLP preprocessing and LDA modeling (.mm .txt etc)
support_path='/Users/eran/Galvanize_more_repositories/Support_group_MD/' # the main directory of the project

In [12]:
dff=pd.read_csv(joinp(pitch_path,'merged_W_id.csv'))

In [13]:
dff=dff.drop(['Unnamed: 0'],axis=1)

In [14]:
dff.head()

,user_id,topic_ID,ID_of_post_in_topic,post_ID,post_text
0,0,1,0,100,HiIs coconut nectar and coconut butter ok to h...
1,1,1,1,101,Not sure about coconut butter. Coconut nectar ...
2,0,1,2,102,Thank you! I have not had any reactions to the...
3,2,2,0,200,"Ok, so first thing I need to know because I ha..."
4,3,2,1,201,"First, wanna say it’s nice to see a post by a ..."


In [17]:
import spacy
import pandas as pd
import numpy as np
import itertools as it

nlp = spacy.load('en')

In [18]:
post8=dff['post_text'][8]
print('Post8:')
print(post8)
type(post8) ## nlp in spaCy can only get string 

Post8:
HelloI have been on this diet for 6 months with no change and have been using Nystatin orally and via enema for 6 weeks now (with good affect) I still feel like something is missing though. My tongue remains white despite using very high dose nystatin. I was wondering if anyone had tried doxycycline and fluconazole? I only ask because all the stool and swab tests I’ve had done come back positive to staph infection and negative for candida I’m thinking it’s an underlying staph infection that feeds the candida. I have been treating this issue for 2 years now


str

In [19]:
parsed_post= nlp(post8)

In [20]:
parsed_post ## note that "nothing happend" cause the magic is under the hood

HelloI have been on this diet for 6 months with no change and have been using Nystatin orally and via enema for 6 weeks now (with good affect) I still feel like something is missing though. My tongue remains white despite using very high dose nystatin. I was wondering if anyone had tried doxycycline and fluconazole? I only ask because all the stool and swab tests I’ve had done come back positive to staph infection and negative for candida I’m thinking it’s an underlying staph infection that feeds the candida. I have been treating this issue for 2 years now

In [21]:
for num, sentence in enumerate(parsed_post.sents):
    print('Sent{}:'.format(num + 1))
    print(sentence)
    print('')

Sent1:
HelloI have been on this diet for 6 months with no change and have been using Nystatin orally and via enema for 6 weeks now (with good affect)

Sent2:
I still feel like something is missing though.

Sent3:
My tongue remains white despite using very high dose nystatin.

Sent4:
I was wondering if anyone had tried doxycycline and fluconazole?

Sent5:
I only ask because all the stool and swab tests I’ve had done come back positive to staph infection and negative for candida

Sent6:
I’m thinking it’s an underlying staph infection that feeds the candida.

Sent7:
I have been treating this issue for 2 years now



In [22]:
# for some reason this gives back the entities only in the 1st sentence
for num, entity in enumerate(parsed_post.ents):
    print('Entity {}:'.format(num + 1), entity, '-', entity.label_)
    print ('')

Entity 1: 6 months - DATE

Entity 2: Nystatin - PERSON

Entity 3: 6 weeks - DATE

Entity 4: 2 years - DATE



In [23]:
token_text = [token.orth_ for token in parsed_post]
token_pos = [token.pos_ for token in parsed_post]

pd.DataFrame({'token_text':token_text,'part_of_speech':token_pos}).head(7)

,part_of_speech,token_text
0,ADJ,HelloI
1,VERB,have
2,VERB,been
3,ADP,on
4,DET,this
5,NOUN,diet
6,ADP,for


In [24]:
# lematizing(steming) i.e. normalizing and 
token_lemma = [token.lemma_ for token in parsed_post]
token_shape = [token.shape_ for token in parsed_post]

pd.DataFrame(list(zip(token_text, token_lemma, token_shape)), ## using list because otherwise zip causes an error
             columns=['token_text', 'token_lemma', 'token_shape']).head(10)

,token_text,token_lemma,token_shape
0,HelloI,helloi,XxxxxX
1,have,have,xxxx
2,been,be,xxxx
3,on,on,xx
4,this,this,xxxx
5,diet,diet,xxxx
6,for,for,xxx
7,6,6,d
8,months,month,xxxx
9,with,with,xxxx


In [25]:
token_entity_type = [token.ent_type_ for token in parsed_post]
token_entity_iob = [token.ent_iob_ for token in parsed_post]

pd.DataFrame(list(zip(token_text, token_entity_type, token_entity_iob)),
             columns=['token_text', 'entity_type', 'inside_outside_begin']).head(8)

,token_text,entity_type,inside_outside_begin
0,HelloI,,O
1,have,,O
2,been,,O
3,on,,O
4,this,,O
5,diet,,O
6,for,,O
7,6,DATE,B


What about a variety of other token-level attributes, such as the relative frequency of tokens, and whether or not a token matches any of these categories?
- stopword
- punctuation
- whitespace
- represents a number
- whether or not the token is included in spaCy's default vocabulary?

In [26]:

token_attributes = [(token.orth_,
                     token.prob,
                     token.is_stop,
                     token.is_punct,
                     token.is_space,
                     token.like_num,
                     token.is_oov)
                    for token in parsed_post]

df = pd.DataFrame(token_attributes,
                  columns=['text',
                           'log_probability',
                           'stop?',
                           'punctuation?',
                           'whitespace?',
                           'number?',
                           'out of vocab.?'])

df.loc[:, 'stop?':'out of vocab.?'] = (df.loc[:, 'stop?':'out of vocab.?']
                                       .applymap(lambda x: u'Yes' if x else u''))

print("There is a problem with out of vocabulary column..")
df.head(8)

There is a problem with out of vocabulary column..


,text,log_probability,stop?,punctuation?,whitespace?,number?,out of vocab.?
0,HelloI,-20.0,,,,,Yes
1,have,-20.0,Yes,,,,Yes
2,been,-20.0,Yes,,,,Yes
3,on,-20.0,Yes,,,,Yes
4,this,-20.0,Yes,,,,Yes
5,diet,-20.0,,,,,Yes
6,for,-20.0,Yes,,,,Yes
7,6,-20.0,,,,Yes,Yes


In [27]:
### The end (for now)